In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import os, sys
sys.path.insert(0, os.path.abspath(".."))
from utilidades.calibration import utilities as ult

import sklearn.pipeline

(DONE) * Lightgbm       - Algorítimico 
(DONE) * Smote          - Preprocessing

- Vini
* CDBH           - Híbrido
* OSM Classifier - Algorítimico

(DONE)* SVM VR         - Preprocessing 

(STARTING)* DBSMOTE        - Preprocessing


- Manaus
(DOING)* HDDT Emsemble  - Algorítimico
(DOING)* WHMBoost       - Cost Sensitivity
* CHMDT          - Cost Sensitivity 
* CCR            - Preprocessing
* EHSO           - Preprocessing

(DONE) * EasyEmsemble   - Algorítimico
* Método novo    - Algorítimico

In [53]:
df = pd.read_csv('breast-cancer.csv') ; print(df.shape,df.select_dtypes('object').value_counts())

(569, 32) diagnosis
B            357
M            212
dtype: int64


In [5]:
try:
    df.drop('id',axis=1, inplace=True)
except:
    pass
df.diagnosis.replace('B',0,inplace=True) # Benignos para 0
df.diagnosis.replace('M',1,inplace=True) # Malignos para 1

In [65]:
import skopt
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler

X_ = df.drop('diagnosis', axis=1)
y = df[['diagnosis']]

X = np.asarray(X_)
y = np.asarray(y).ravel()

X_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=32,stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


SVM_vr_search = BayesSearchCV(
    SVC(),
    {
        'C': Integer(5,10),
        'gamma': Integer(5,10),
        'degree': Integer(1,8),
        'kernel': Categorical(['linear', 'poly', 'rbf','sigmoid'])
    },
     n_iter=32,
     random_state=32,
     cv=5,
     scoring='accuracy',
     return_train_score=True
)

SVM_vr_search.fit(X_train_scaled, y_train)

X_test_scaled = scaler.transform(x_test)
print(SVM_vr_search.score(X_test_scaled, y_test))

results_cv = pd.DataFrame(SVM_vr_search.cv_results_)

temp = results_cv[['mean_train_score', 'mean_test_score']]
temp['diff'] = temp['mean_test_score'] - temp['mean_train_score']
to_go = temp[abs(temp['diff']) < 0.05].sort_values(by = 'mean_test_score', ascending = False).head(1).index

params = results_cv.loc[to_go.values[0]]
kwargs = params.params
print(kwargs)

best_SVM = SVC(random_state=32,**kwargs).fit(X_train_scaled,y_train)
X_testz = scaler.inverse_transform(X_train_scaled)
new_train = pd.DataFrame(X_testz,columns=X_.columns)
nem_train['y'] = y_train2

c:\Users\vinic\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\vinic\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


0.9574468085106383
OrderedDict([('C', 6), ('degree', 1), ('gamma', 7), ('kernel', 'linear')])


C:\Users\vinic\AppData\Local\Temp/ipykernel_6820/1822791052.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['diff'] = temp['mean_test_score'] - temp['mean_train_score']


In [68]:
class svm_vr():
    def __init__(self):

        # Importing packages.
        import sklearn as sk
        from sklearn.svm import SVC
        from sklearn.model_selection import train_test_split
        import seaborn as sns
        from sklearn.model_selection import GridSearchCV
        import optuna 
        from optuna.samplers import TPESampler
        from sklearn.model_selection import cross_val_score
        import time
        import skopt
        from skopt import BayesSearchCV

    def svm_vr(Y,df,random_state=32):
        """ This function fits a SVM Classifier, taking the best result from the grid searching.
        After the best fit has been achieved, the training dependent variable values must be replaced by the predicts of the trained model, thus obtaining the balancing.

        Following the instructions of the Authors.
        They suggest using higher values for C and the RBF kernel function.

        "The proposed methodology followed a two phase
        approach. During the first phase the available training data is used
        to train SVM. Later, the target values of the training data are replaced
        by the corresponding predictions of the trained SVM. During the
        second phase, this modified data is used to train MLP, LR, and RF
        separately" Authors.

        Based on the ideia by the authors, we select the best model, by the value of accuracy
        """
        # Setting the X and y
        X = df.drop(Y, axis=1)
        y = df[[Y]]

        X = X.

        # Split the data
        X_train, x_test,
        y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=random_state)

        SVM_vr_search = BayesSearchCV(
            SVC(),
            {
                'C': Integer(5,10),
                'gamma': Integer(5,10),
                'degree': Integer(1,8),
                'kernel': Categorical(['linear', 'poly', 'rbf','sigmoid'])
            },
            n_iter=32,
            random_state=32,
            cv=5,
            scoring='accuracy',
            return_train_score=True
        )
        try:
            
            SVM_vr_search.fit(X_train, y_train)
            
        except ValueError:

            # Transforming to StandardScaler:
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            SVM_vr_search.fit(X_train_scaled, y_train)

            # Fitting the scalled
            X_test_scaled = scaler.transform(x_test)
            SVM_vr_search.score(X_test_scaled, y_test)

        results_cv = pd.DataFrame(SVM_vr_search.cv_results_)
        
        temp = results_cv[['mean_train_score', 'mean_test_score']]
        temp['diff'] = temp['mean_test_score'] - temp['mean_train_score']
        to_go = temp[abs(temp['diff']) < 0.05].sort_values(by = 'mean_test_score', ascending = False).head(1).index
        
        params = results_cv.loc[to_go.values[0]]
        kwargs = params.params
        print(kwargs)

        best_SVM = SVC(random_state=32,**kwargs).fit(X)
        y_train2= best_SVM.predict(X_train_scaled)
        X_trainz = scaler.inverse_transform(X_train_scaled)
        new_train = pd.DataFrame(X_trainz,columns=X_.columns)
        new_train['y'] = y_train2 ; new_train
        
        

